**Clinical Reasoning with GPT-3**

**Load Necessary Dependencies**

In [ ]:
import pandas as pd
import cohere
from datasets import load_dataset
import openai
import os
import dsp
import pandas as pd
import numpy as np

In [ ]:
try:
    import datasets
    root_path = '.'
except ModuleNotFoundError:
    !git clone https://github.com/cgpotts/cs224u/
    !pip install -r cs224u/requirements.txt
    root_path = 'dsp'

In [ ]:
key = ## API key ##
os.environ["DSP_NOTEBOOK_CACHEDIR"] = os.path.join(root_path, 'cache')
openai_key = key

**Load and clean questions**

In [ ]:
test = pd.read_json("/content/test.jsonl", lines=True)

In [ ]:
test = test[test['meta_info']!= 'step1']

In [ ]:
test_orig = test.reset_index(drop=True)

In [ ]:
test_orig['question'] = test_orig['question'].str.replace('Which of the following', 'What')
test_orig['question'] = test_orig['question'].str.replace('which of the following', 'what')

In [ ]:
test_orig.to_csv('dev_cases.csv', index=False)

**Selected Diagnosis Task Questions**

In [ ]:
diag_q = pd.read_csv('test_question_set.csv')

Load DSP module and Openai API with key
(Note must provide own key)

In [ ]:
lm = dsp.GPT3(model='text-davinci-003', api_key=key)
dsp.settings.configure(lm=lm)

In [ ]:
Question = dsp.Type(prefix="Question:", desc="${A patient passage to be analyzed and determine the patient's diagnosis}")
Answer = dsp.Type(prefix="Answer:", desc="${a short factoid diagnosis, often between 1 and 9 words}", format=dsp.format_answers)
qa_template = dsp.Template(instructions="Diagnose the patients condition using the given passage with a short factoid answer.", question=Question(), answer=Answer())

**Diagnosis: Few Shot with Chain of Thought (CoT)**

In [ ]:
demo_example_CoT = [{'question':'Shortly after undergoing a bipolar prosthesis for a displaced femoral neck fracture of the left hip acquired after a fall the day before, an 80-year-old woman suddenly develops dyspnea. The surgery under general anesthesia with sevoflurane was uneventful, lasting 98 minutes, during which the patient maintained oxygen saturation readings of 96â€“100% on 8 L of oxygen. She has a history of hypertension, osteoporosis, and osteoarthritis of her right knee. Her medications include ramipril, naproxen, ranitidine, and a multivitamin. She appears cyanotic, drowsy, and is oriented only to person. Her temperature is 38.6Â°C (101.5Â°F), pulse is 135/minute, respirations are 36/min, and blood pressure is 155/95 mm Hg. Pulse oximetry on room air shows an oxygen saturation of 81%. There are several scattered petechiae on the anterior chest wall. Laboratory studies show a hemoglobin concentration of 10.5 g/dL, a leukocyte count of 9,000/mm3, a platelet count of 145,000/mm3, and a creatine kinase of 190 U/L. An ECG shows sinus tachycardia. What is the most likely diagnosis?','answer': 'Fat embolism', 'rationale': 'The patient had a surgical repair of a displaced femoral neck fracture.  The patient has petechiae.  The patient has a new oxygen requirement, meaning they are having difficulty with their breathing.  This patient most likely has a fat embolism'}, {'question': 'A 55-year-old man comes to the emergency department because of a dry cough and severe chest pain beginning that morning. Two months ago, he was diagnosed with inferior wall myocardial infarction and was treated with stent implantation of the right coronary artery. He has a history of hypertension and hypercholesterolemia. His medications include aspirin, clopidogrel, atorvastatin, and enalapril. His temperature is 38.5Â°C (101.3Â°F), pulse is 92/min, respirations are 22/min, and blood pressure is 130/80 mm Hg. Cardiac examination shows a high-pitched scratching sound best heard while sitting upright and during expiration. The remainder of the examination shows no abnormalities. An ECG shows diffuse ST elevations. Serum studies show a troponin I of 0.005 ng/mL (N < 0.01). What is the most likely cause of this patients symptoms?', 'answer': 'Dresslers Syndrome', 'rationale': 'This patient is having chest pain.  They recently had a heart attack and have new chest pain, suggestig he may have a problem with his heart.  The EKG has diffuse ST elevations and scratching murmur.  This patient likely has Dressler Syndrome.'}]


In [ ]:
Rationale = dsp.Type(
    prefix="Rationale: a step-by-step deduction that identifies the correct response.",
    desc="${Provide a step by step deduction that identifies the correct response.}"
)
Question = dsp.Type(prefix="Question:", desc="${the question to be answered}")
Answer = dsp.Type(prefix="Answer:", desc="${a short factoid answer, often between 1 and 5 words}", format=dsp.format_answers)

In [ ]:
qa_template_with_CoT = dsp.Template(
    instructions=qa_template.instructions, question=Question(), rationale=Rationale(), answer=Answer()
)

In [ ]:
diag_q['SC'] = np.nan
diag_q['SC__rationale'] = np.nan

In [ ]:
@dsp.transformation
def QA_predict(example: dsp.Example, sc=True):
    rats = [ ]
    if sc == True:
        example, completions = dsp.generate(qa_template_with_CoT, n=23, temperature=0.7)(example, stage='qa')

        for i in range(23):
          try:
            rats.append(completions[i].rationale + '__')
          except IndexError:
            break
        # print(rats)
        completions_total = dsp.majority(completions)
    else:
        example, completions = dsp.generate(qa_template_with_CoT)(example, stage='qa')
        #print(completions.rationale)
    return example.copy(answer=completions_total.answer, rationale = rats)

In [ ]:
def vanilla_LM_QA_FewCoT_SC(question: str) -> str:
    example = dsp.Example(question=question, demos=demo_example_CoT)
    return QA_predict(example, sc=True).answer, QA_predict(example, sc=True).rationale

In [ ]:
for i in range(519):
   print(i)
   a = vanilla_LM_QA_FewCoT_SC(diag_q['question'][i])
   diag_q['SC'][i] = a[0]
   diag_q['SC__rationale'][i] = a[1]
   diag_q.to_csv('/content/drive/MyDrive/file1.csv', index=False)

In [ ]:
diag_q.to_csv('SC.csv', index=False)

**Clinical Reasoning Prompts**


**Clinical Reasoning**

Here we use a modified CoT for Self-consistency.  Below are the different rationales for the different types of questions:
- Differential Diagnosis
- Intuitive Reasoning
- Analytic Reasoning
- Bayesian Inference

In [ ]:
diag_q['diag_IR'] = np.nan
diag_q['diag_IR_rationale'] = np.nan
diag_q['IR_numb'] = np.nan
diag_q['diag_AR'] = np.nan
diag_q['diag_AR_rationale'] = np.nan
diag_q['AR_numb'] = np.nan
diag_q['diag_BR'] = np.nan
diag_q['diag_BR_rationale'] = np.nan
diag_q['BR_numb'] = np.nan
diag_q['diag_DR'] = np.nan
diag_q['diag_DR_rationale'] = np.nan
diag_q['DR_numb'] = np.nan


In [ ]:
Context = dsp.Type(
    prefix="Context:\n",
    desc="${sources that may contain relevant content}",
    format=dsp.passages2text
)

Examples for each type of reasoning:

In [ ]:
demo_example_DR = [{'question':'Shortly after undergoing a bipolar prosthesis for a displaced femoral neck fracture of the left hip acquired after a fall the day before, an 80-year-old woman suddenly develops dyspnea. The surgery under general anesthesia with sevoflurane was uneventful, lasting 98 minutes, during which the patient maintained oxygen saturation readings of 96â€“100% on 8 L of oxygen. She has a history of hypertension, osteoporosis, and osteoarthritis of her right knee. Her medications include ramipril, naproxen, ranitidine, and a multivitamin. She appears cyanotic, drowsy, and is oriented only to person. Her temperature is 38.6Â°C (101.5Â°F), pulse is 135/minute, respirations are 36/min, and blood pressure is 155/95 mm Hg. Pulse oximetry on room air shows an oxygen saturation of 81%. There are several scattered petechiae on the anterior chest wall. Laboratory studies show a hemoglobin concentration of 10.5 g/dL, a leukocyte count of 9,000/mm3, a platelet count of 145,000/mm3, and a creatine kinase of 190 U/L. An ECG shows sinus tachycardia. What is the most likely diagnosis?','answer': 'Fat embolism', 'rationale': 'This patient has shortness of breath after a long bone surgery.  The differential for this patient is pulmonary embolism, fat embolism, myocardial infarction, blood loss, anaphylaxis, or a drug reaction.  The patient has petechiae which makes fat embolism more likely.  This patient most likely has a fat embolism.'}, {'question': 'A 55-year-old man comes to the emergency department because of a dry cough and severe chest pain beginning that morning. Two months ago, he was diagnosed with inferior wall myocardial infarction and was treated with stent implantation of the right coronary artery. He has a history of hypertension and hypercholesterolemia. His medications include aspirin, clopidogrel, atorvastatin, and enalapril. His temperature is 38.5Â°C (101.3Â°F), pulse is 92/min, respirations are 22/min, and blood pressure is 130/80 mm Hg. Cardiac examination shows a high-pitched scratching sound best heard while sitting upright and during expiration. The remainder of the examination shows no abnormalities. An ECG shows diffuse ST elevations. Serum studies show a troponin I of 0.005 ng/mL (N < 0.01). What is the most likely cause of this patients symptoms?', 'answer': 'Dresslers Syndrome', 'rationale': 'This patient has chest pain with diffuse ST elevations after a recent myocardial infarction.  The differential for this patient includes: myocardial infarction, pulmonary embolism, pericarditis, Dressler syndrome, aortic dissection, and costochondritis.   This patient likely has a high-pitched scratching sound on auscultation associated with pericarditis and Dressler Syndrome.  This patient has diffuse ST elevations associated with Dressler Syndrome.  This patient most likely has Dressler Syndrome.'}]
demo_example_IR = [{'question':'Shortly after undergoing a bipolar prosthesis for a displaced femoral neck fracture of the left hip acquired after a fall the day before, an 80-year-old woman suddenly develops dyspnea. The surgery under general anesthesia with sevoflurane was uneventful, lasting 98 minutes, during which the patient maintained oxygen saturation readings of 96â€“100% on 8 L of oxygen. She has a history of hypertension, osteoporosis, and osteoarthritis of her right knee. Her medications include ramipril, naproxen, ranitidine, and a multivitamin. She appears cyanotic, drowsy, and is oriented only to person. Her temperature is 38.6Â°C (101.5Â°F), pulse is 135/minute, respirations are 36/min, and blood pressure is 155/95 mm Hg. Pulse oximetry on room air shows an oxygen saturation of 81%. There are several scattered petechiae on the anterior chest wall. Laboratory studies show a hemoglobin concentration of 10.5 g/dL, a leukocyte count of 9,000/mm3, a platelet count of 145,000/mm3, and a creatine kinase of 190 U/L. An ECG shows sinus tachycardia. What is the most likely diagnosis?','answer': 'Fat embolism', 'rationale': 'This patient has findings of petechiae, altered mental status, shortness of breath, and recent surgery suggesting a diagnosis of fat emboli.  The patient most likely has a fat embolism. '}, {'question': 'A 55-year-old man comes to the emergency department because of a dry cough and severe chest pain beginning that morning. Two months ago, he was diagnosed with inferior wall myocardial infarction and was treated with stent implantation of the right coronary artery. He has a history of hypertension and hypercholesterolemia. His medications include aspirin, clopidogrel, atorvastatin, and enalapril. His temperature is 38.5Â°C (101.3Â°F), pulse is 92/min, respirations are 22/min, and blood pressure is 130/80 mm Hg. Cardiac examination shows a high-pitched scratching sound best heard while sitting upright and during expiration. The remainder of the examination shows no abnormalities. An ECG shows diffuse ST elevations. Serum studies show a troponin I of 0.005 ng/mL (N < 0.01). What is the most likely cause of this patients symptoms?', 'answer': 'Dresslers Syndrome', 'rationale': 'This patient had a recent myocardial infarction with new development of diffuse ST elevations, chest pain, and a high pitched scratching murmur which are found in Dresslers syndrome.   This patient likely has Dresslers Syndrome.'}]
demo_example_AR = [{'question':'Shortly after undergoing a bipolar prosthesis for a displaced femoral neck fracture of the left hip acquired after a fall the day before, an 80-year-old woman suddenly develops dyspnea. The surgery under general anesthesia with sevoflurane was uneventful, lasting 98 minutes, during which the patient maintained oxygen saturation readings of 96â€“100% on 8 L of oxygen. She has a history of hypertension, osteoporosis, and osteoarthritis of her right knee. Her medications include ramipril, naproxen, ranitidine, and a multivitamin. She appears cyanotic, drowsy, and is oriented only to person. Her temperature is 38.6Â°C (101.5Â°F), pulse is 135/minute, respirations are 36/min, and blood pressure is 155/95 mm Hg. Pulse oximetry on room air shows an oxygen saturation of 81%. There are several scattered petechiae on the anterior chest wall. Laboratory studies show a hemoglobin concentration of 10.5 g/dL, a leukocyte count of 9,000/mm3, a platelet count of 145,000/mm3, and a creatine kinase of 190 U/L. An ECG shows sinus tachycardia. What is the most likely diagnosis?','answer': 'Fat embolism', 'rationale': 'The patient recently had large bone surgery making fat emboli a potential cause because the bone marrow was manipulated.  Petechiae can form in response to capillary inflammation caused by fat emboli.  Fat micro globules cause CNS microcirculation occlusion causing confusion and altered mental status.  Fat obstruction in the pulmonary arteries can cause tachycardia and shortness of breath as seen in this patient.   This patient most likely has a fat embolism.'}, {'question': 'A 55-year-old man comes to the emergency department because of a dry cough and severe chest pain beginning that morning. Two months ago, he was diagnosed with inferior wall myocardial infarction and was treated with stent implantation of the right coronary artery. He has a history of hypertension and hypercholesterolemia. His medications include aspirin, clopidogrel, atorvastatin, and enalapril. His temperature is 38.5Â°C (101.3Â°F), pulse is 92/min, respirations are 22/min, and blood pressure is 130/80 mm Hg. Cardiac examination shows a high-pitched scratching sound best heard while sitting upright and during expiration. The remainder of the examination shows no abnormalities. An ECG shows diffuse ST elevations. Serum studies show a troponin I of 0.005 ng/mL (N < 0.01). What is the most likely cause of this patients symptoms?', 'answer': 'Dresslers Syndrome', 'rationale': 'This patient had a recent myocardial infarction which can cause myocardial inflammation that causes pericarditis and Dressler Syndrome.  The diffuse ST elevations and high pitched scratching murmur can be signs of pericardial inflammation as the inflamed pericardium rubs against the pleura as seen with Dressler Syndrome.  This patient likely has Dressler Syndrome.'}]
demo_example_BR = [{'question':'Shortly after undergoing a bipolar prosthesis for a displaced femoral neck fracture of the left hip acquired after a fall the day before, an 80-year-old woman suddenly develops dyspnea. The surgery under general anesthesia with sevoflurane was uneventful, lasting 98 minutes, during which the patient maintained oxygen saturation readings of 96â€“100% on 8 L of oxygen. She has a history of hypertension, osteoporosis, and osteoarthritis of her right knee. Her medications include ramipril, naproxen, ranitidine, and a multivitamin. She appears cyanotic, drowsy, and is oriented only to person. Her temperature is 38.6Â°C (101.5Â°F), pulse is 135/minute, respirations are 36/min, and blood pressure is 155/95 mm Hg. Pulse oximetry on room air shows an oxygen saturation of 81%. There are several scattered petechiae on the anterior chest wall. Laboratory studies show a hemoglobin concentration of 10.5 g/dL, a leukocyte count of 9,000/mm3, a platelet count of 145,000/mm3, and a creatine kinase of 190 U/L. An ECG shows sinus tachycardia. What is the most likely diagnosis?','answer': 'Fat embolism', 'rationale': 'The prior probability of fat embolism is 0.05% however the patient has petechiae on exam which is seen with fat emboli, which increases the posterior probability of fat embolism to 5%.  Altered mental status increases the probability further to 10%.  Recent orthopedic surgery increases the probability of fat emboli syndrome to 60%.  This patient most likely has a fat embolism. '}, {'question': 'A 55-year-old man comes to the emergency department because of a dry cough and severe chest pain beginning that morning. Two months ago, he was diagnosed with inferior wall myocardial infarction and was treated with stent implantation of the right coronary artery. He has a history of hypertension and hypercholesterolemia. His medications include aspirin, clopidogrel, atorvastatin, and enalapril. His temperature is 38.5Â°C (101.3Â°F), pulse is 92/min, respirations are 22/min, and blood pressure is 130/80 mm Hg. Cardiac examination shows a high-pitched scratching sound best heard while sitting upright and during expiration. The remainder of the examination shows no abnormalities. An ECG shows diffuse ST elevations. Serum studies show a troponin I of 0.005 ng/mL (N < 0.01). What is the most likely cause of this patients symptoms?', 'answer': 'Dresslers Syndrome', 'rationale': 'The prior probability of Dressler Syndrome is 0.01%. The patient has diffuse ST elevations, increasing the probability of Dressler Syndrome to 5%.  The patient has a scratching murmur which increases the probability to 10%.  In the setting of a recent MI the posterior probability of myocardial infarction is 55%.  This patient likely has Dressler Syndrome.'}]

In [ ]:
# Intuitive Rationale
Rationale_diag_IR = dsp.Type(
    prefix="Rationale: a step-by-step deduction that identifies the correct response",
    desc="${Use symptom, signs and laboratory disease associations to step by step deduce the correct response}")

# Differential Rationale
Rationale_diag_DR = dsp.Type(
    prefix="Rationale: a step-by-step deduction that identifies the correct response",
    desc="${Use step by step deduction to create a differential diagnosis and then use step by step deduction to determine the correct response.}")

# Analytic Rationale
Rationale_diag_AR = dsp.Type(
    prefix="Rationale: a step-by-step deduction that identifies the correct response.",
    desc="${Use analytic reasoning to deduce the physiologic or biochemical pathophysiology of the patient and step by step identify the correct response.}")

# Bayesian Rationale
Rationale_diag_BR = dsp.Type(
    prefix="Rationale: a step-by-step deduction that identifies the correct response.",
    desc="${Use step by step Bayesian Inference to create a prior probability that is updated with new information in the history to produce a posterior probability and determine the final diagnosis.}")



In [ ]:
 qa_template_diag_IR = dsp.Template(
    instructions=qa_template.instructions, question=Question(), rationale=Rationale_diag_IR(), answer=Answer()
)

 qa_template_diag_AR = dsp.Template(
    instructions=qa_template.instructions, question=Question(), rationale=Rationale_diag_AR(), answer=Answer()
)


 qa_template_diag_BR = dsp.Template(
    instructions=qa_template.instructions,question=Question(), rationale=Rationale_diag_BR(), answer=Answer()
)

 qa_template_diag_DR = dsp.Template(
    instructions=qa_template.instructions,question=Question(), rationale=Rationale_diag_DR(), answer=Answer()
)


In [ ]:
@dsp.transformation
def QA_predict_IR(example: dsp.Example, sc=True):
    rats = [ ]
    if sc ==True :
        example, completions = dsp.generate(qa_template_diag_IR, n=23, temperature=0.7)(example, stage='qa')
        runs = len(completions)
        for i in range(len(completions)):
          rats.append(completions[i].rationale + '__')
        completions_total = dsp.majority(completions)
    else:
        example, completions = dsp.generate(qa_template_diag_IR)(example, stage='qa')
    return example.copy(answer=completions_total.answer, rationale = rats, run_numb = runs)

def QA_predict_AR(example: dsp.Example, sc=True):
    rats = [ ]
    if sc ==True :
        example, completions = dsp.generate(qa_template_diag_AR, n=23, temperature=0.7)(example, stage='qa')
        runs = len(completions)
        for i in range(len(completions)):
          rats.append(completions[i].rationale + '__')
        completions_total = dsp.majority(completions)
    else:
        example, completions = dsp.generate(qa_template_diag_AR)(example, stage='qa')
    return example.copy(answer=completions_total.answer, rationale = rats, run_numb = runs)

def QA_predict_BR(example: dsp.Example, sc=True):
    rats = [ ]
    if sc ==True :
        example, completions = dsp.generate(qa_template_diag_BR, n=23, temperature=0.7)(example, stage='qa')
        runs = len(completions)
        for i in range(len(completions)):
          rats.append(completions[i].rationale + '__')
        completions_total = dsp.majority(completions)
    else:
        example, completions = dsp.generate(qa_template_diag_BR)(example, stage='qa')
    return example.copy(answer=completions_total.answer, rationale = rats, run_numb = runs)

def QA_predict_DR(example: dsp.Example, sc=True):
    rats = [ ]
    if sc ==True :
        example, completions = dsp.generate(qa_template_diag_DR, n=23, temperature=0.7)(example, stage='qa')
        runs = len(completions)
        for i in range(len(completions)):
          rats.append(completions[i].rationale + '__')
        completions_total = dsp.majority(completions)
    else:
        example, completions = dsp.generate(qa_template_diag_DR)(example, stage='qa')
    return example.copy(answer=completions_total.answer, rationale = rats, run_numb = runs)


In [ ]:
import random
def retrieve_then_QA_IR(question: str, selected) -> str:
    demos = demo_example_IR
    example = dsp.Example(question=question, demos=demos)
    return QA_predict_IR(example, sc=True).answer, QA_predict_IR(example, sc=True).rationale, QA_predict_IR(example, sc=True).run_numb


def retrieve_then_QA_AR(question: str, selected) -> str:
    demos = demo_example_AR
    example = dsp.Example(question=question, context=passages, demos=demos)

    return QA_predict_AR(example, sc=True).answer, QA_predict_AR(example, sc=True).rationale, QA_predict_AR(example, sc=True).run_numb

def retrieve_then_QA_BR(question: str, selected) -> str:
    demos = demo_example_BR
    example = dsp.Example(question=question, context=passages, demos=demos)

    return QA_predict_BR(example, sc=True).answer, QA_predict_BR(example, sc=True).rationale, QA_predict_BR(example, sc=True).run_numb


def retrieve_then_QA_DR(question: str, selected) -> str:
    demos = demo_example_DR
    example = dsp.Example(question=question, context=passages, demos=demos)

    return QA_predict_DR(example, sc=True).answer, QA_predict_DR(example, sc=True).rationale, QA_predict_DR(example, sc=True).run_numb



In [ ]:
for i in range(519):
  print(i)
  a = retrieve_then_QA_IR(diag_q['question'][i],selected)
  diag_q['diag_IR'][i] = a[0]
  diag_q['diag_IR_rationale'][i] = a[1]
  diag_q['IR_numb'][i] = a[2]

  b = retrieve_then_QA_DR(diag_q['question'][i],selected)
  diag_q['diag_DR5'][i] = b[0]
  diag_q['diag_DR_rationale'][i] = b[1]
  diag_q['DR_numb'][i] = b[2]

  c = retrieve_then_QA_AR(diag_q['question'][i],selected)
  diag_q['diag_AR6'][i] = c[0]
  diag_q['diag_AR_rationale'][i] = c[1]
  diag_q['AR_numb'][i] = c[2]

  d = retrieve_then_QA_BR(diag_q['question'][i],selected)
  diag_q['diag_BR4'][i] = d[0]
  diag_q['diag_BR_rationale'][i] = d[1]
  diag_q['BR_numb'][i] = d[2]



In [ ]:
diag_q.to_csv('Test_Results.csv', index=False)
